<!-- Stand-alone notebook? -->
<!-- (Need preprocess #if tests here since they are to be executed -->
<!-- when preprocess is run to get the right #include statements. -->
<!-- Thereafter, mako is executed and we can also have %if syntax.) -->


<!-- Need some intro about the variational formulation -->

# The Poisson equation

## Mathematical problem formulation

We want to solve

<!-- Equation labels as ordinary links -->
<div id="ftut:poisson1"></div>

$$
\begin{equation}
- \nabla^2 u(\boldsymbol{x}) = f(\boldsymbol{x}),\quad \boldsymbol{x}\mbox{ in } \Omega,
\label{ftut:poisson1} \tag{1}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="ftut:poisson1:bc"></div>

$$
\begin{equation}  
u(\boldsymbol{x}) = u_0(\boldsymbol{x}),\quad \boldsymbol{x}\mbox{ on } \partial \Omega{\thinspace .} \label{ftut:poisson1:bc} \tag{2}
\end{equation}
$$

where
$u = u(\boldsymbol{x})$ is the unknown function, $f = f(\boldsymbol{x})$ is a
prescribed function, $\nabla^2$ is the Laplace operator (also
often written as $\Delta$), $\Omega$ is the spatial domain, and
$\partial\Omega$ is the boundary of $\Omega$. A stationary PDE like
this, together with a complete set of boundary conditions, constitute
a *boundary-value problem*, which must be precisely stated before
it makes sense to start solving it with FEniCS.

In two space dimensions with coordinates $x$ and $y$, we can write out
the Poisson equation as

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
- {\partial^2 u\over\partial x^2} -
{\partial^2 u\over\partial y^2} = f(x,y){\thinspace .}
\label{_auto1} \tag{3}
\end{equation}
$$

The unknown $u$ is now a function of two variables, $u = u(x,y)$, defined
over a two-dimensional domain $\Omega$.

The variational formulation of this problem goes as follows:
Find $u \in V$ such that

<!-- Equation labels as ordinary links -->
<div id="ftut:poisson1:var"></div>

$$
\begin{equation} \label{ftut:poisson1:var} \tag{4}
  \int_{\Omega} \nabla u \cdot \nabla v {\, \mathrm{d}x} =
  \int_{\Omega} fv {\, \mathrm{d}x}
  \quad \forall v \in \hat{V}
\end{equation}
$$

where $V$ and $\hat V$ are function spaces for the trial function $u$ and
the test functions $v$, respectively. We shall now take these spaces to
be finite dimensional, described by a mesh containing finite elements.

It is common to write the variational formulation on the form

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}
a(u, v) = L(v){\thinspace .}
\label{_auto2} \tag{5}
\end{equation}
$$

For the Poisson equation, we have:

<!-- Equation labels as ordinary links -->
<div id="ftut:poisson1:vard:a"></div>

$$
\begin{equation}
a(u, v) = \int_{\Omega} \nabla u \cdot \nabla v {\, \mathrm{d}x},
\label{ftut:poisson1:vard:a} \tag{6}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="ftut:poisson1:vard:L"></div>

$$
\begin{equation}  
L(v) = \int_{\Omega} fv {\, \mathrm{d}x}{\thinspace .}  \label{ftut:poisson1:vard:L} \tag{7}
\end{equation}
$$

FEniCS provides all the necessary mathematical notation needed to
express the variational problem $a(u, v) = L(v)$. To solve a linear
PDE in FEniCS, such as the Poisson equation, a user thus needs to
perform only two steps:

  * Express the PDE as a (discrete) variational problem: find $u\in V$
    such that $a(u,v) = L(v)$ for all $v\in \hat{V}$.

  * Choose the finite element spaces $V$ and $\hat V$ by specifying
    the domain (the mesh) and the type of function space (polynomial
    degree and type).

## Choosing a test problem
<div id="ftut:poisson1:testproblem"></div>

The Poisson equation [(1)](#ftut:poisson1) has so far featured a general
domain $\Omega$ and general functions $u_0$ and $f$. For our first
implementation, we must decide on specific choices of $\Omega$, $u_0$,
and $f$.  It will be wise to construct a specific problem where we can
easily check that the computed solution is correct. Solutions that are
lower-order polynomials are primary candidates. Standard finite
element function spaces of degree $r$ will exactly reproduce
polynomials of degree $r$. And piecewise linear elements ($r=1$) are
able to exactly reproduce a quadratic polynomial on a uniformly
partitioned mesh. This important result can be used to verify our
implementation. We just manufacture some quadratic function in 2D as
the exact solution, say

<!-- Equation labels as ordinary links -->
<div id="ftut:poisson1:impl:uex"></div>

$$
\begin{equation}
\label{ftut:poisson1:impl:uex} \tag{8}
{u_{\small\mbox{e}}}(x,y) = 1 +x^2 + 2y^2{\thinspace .}
\end{equation}
$$

By inserting [(8)](#ftut:poisson1:impl:uex) into the Poisson equation
[(1)](#ftut:poisson1), we find that ${u_{\small\mbox{e}}}(x,y)$ is a solution if

$$
f(x,y) = -6,\quad u_0(x,y)={u_{\small\mbox{e}}}(x,y)=1 + x^2 + 2y^2,
$$

regardless of the shape of the domain as long as ${u_{\small\mbox{e}}}$ is prescribed along
the boundary. We choose here, for simplicity,
the domain to be the unit square,

$$
\Omega = [0,1]\times [0,1] {\thinspace .}
$$

This simple but very powerful method for constructing test problems
is called the *method of manufactured solutions*: pick a simple
expression for the exact solution, plug it into the equation to obtain
the right-hand side (source term $f$), then solve the equation with
this right-hand side and try to reproduce the exact solution.

**Tip: Try to verify your code with exact numerical solutions!**

A common approach to testing the implementation of a numerical method
is to compare the numerical
solution with an exact analytical solution of the test problem and
conclude that the program works if the error is "small enough".
Unfortunately, it is impossible to tell if an error of size $10^{-5}$ on a
$20\times 20$ mesh of linear elements is the expected (in)accuracy of the
numerical approximation or if the error also contains the effect of a
bug in the code. All we usually know about the numerical error is its
*asymptotic properties*, for instance that it is proportional to $h^2$
if $h$ is the size of a cell in the mesh. Then we can compare the
error on meshes with different $h$ values to see if the asymptotic
behavior is correct. This is a very powerful verification
technique.
However, if we have a test problem for which
we know that there should be no approximation errors, we know that
the analytical solution of the PDE problem should be reproduced to
machine precision by the program. That is why we emphasize this kind
of test problems throughout this tutorial. Typically, elements of
degree $r$ can reproduce polynomials of degree $r$ exactly, so this
is the starting point for constructing a solution without numerical
approximation errors.



## FEniCS implementation
<div id="ftut:poisson1:impl"></div>
<div id="ftut:poisson1:impl:code"></div>

A FEniCS program for solving our test problem for the Poisson equation
in 2D with the given choices of $u_0$, $f$, and $\Omega$ may look as
follows:

In [1]:
from fenics import *

# Create mesh and define function space
mesh = UnitSquareMesh(8, 8)
V = FunctionSpace(mesh, 'P', 1)

# Define boundary conditions
u0 = Expression('1 + x[0]*x[0] + 2*x[1]*x[1]', degree=1)

def u0_boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, u0, u0_boundary)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(-6.0)
a = dot(grad(u), grad(v))*dx
L = f*v*dx

# Compute solution
u = Function(V)
solve(a == L, u, bc)

# Plot solution on the screen
u.rename('u', 'solution')
plot(u)
plot(mesh)

# Dump solution to file in VTK format
vtkfile = File('poisson.pvd')
vtkfile << u

# Compute and print error
u_e = interpolate(u0, V)
error = max(abs(u_e.vector().array() - u.vector().array()))
import numpy as np
error = np.abs(u_e.vector().array() - u.vector().array()).max()
print('error =', error)

# Hold plot
interactive()

## Dissection of the program
<div id="ftut:poisson1:impl:dissect"></div>


The listed FEniCS program defines a finite element mesh, a finite
element function space $V$ on this mesh, boundary conditions for $u$
(the function $u_0$), and the bilinear and linear forms $a(u,v)$ and
$L(v)$.  Thereafter, the unknown trial function $u$ is computed. Then
we can compare the numerical and exact solution as well as visualize
the computed solution $u$.


### The important first line

The first line in the program,

```Python
        from fenics import *
```

imports the key classes `UnitSquareMesh`, `FunctionSpace`, `Function`,
and so forth, from the FEniCS library.  All FEniCS programs for
solving PDEs by the finite element method normally start with this
line.



### Generating simple meshes

The statement

```Python
        mesh = UnitSquareMesh(8, 8)
```

defines a uniform finite element mesh over the unit square
$[0,1]\times [0,1]$. The mesh consists of *cells*, which in 2D are triangles
with straight sides. The parameters 8 and 8 specify that the square
should be divided into $8\times 8$ rectangles, each divided into a pair of
triangles. The total number of triangles (cells) thus becomes
128. The total number of vertices in the mesh is $9\cdot 9=81$.
In later chapters, you will learn how to generate more complex meshes.

[hpl 1: Note that plot was made by the old partitioning $6\times 4$. Probably
no issue.]



### Defining the finite element function space

Having a mesh, we can define a finite element function space `V` over
this mesh:

```Python
        V = FunctionSpace(mesh, 'P', 1)
```

The second argument `'P`' specifies the type of element, while the third
argument is the degree of the basis functions of the element. The type
of element is here "P", implying the standard Lagrange family of
elements. You may also use `'Lagrange'` to specify this type of
element. FEniCS supports all simplex element families and the notation
defined in the [Periodic Table of the Finite Elements](http://femtable.org) [[ArnoldLogg2014]](#ArnoldLogg2014).


The third argument `1` specifies the degree of the finite element.  In
this case, the standard $\mathsf{P}_1$ linear Lagrange element, which
is a triangle with nodes at the three vertices. Some finite element
practitioners refer to this element as the "linear triangle". The
computed solution $u$ will be continuous and linearly varying in $x$
and $y$ over each cell in the mesh. Higher-degree polynomial
approximations over each cell are trivially obtained by increasing the
third parameter to `FunctionSpace`, which will then generate function
spaces of type $\mathsf{P}_2$, $\mathsf{P}_3$, and so forth.
Changing the second parameter to `'DP'` creates a function
space for discontinuous Galerkin methods.


### Defining the trial and test functions

In mathematics, we distinguish between the trial and test spaces $V$
and $\hat{V}$. The only difference in the present problem is the
boundary conditions. In FEniCS we do not specify the boundary
conditions as part of the function space, so it is sufficient to work
with one common space `V` for the and trial and test functions in the
program:

```Python
        u = TrialFunction(V)
        v = TestFunction(V)
```

### Defining the boundary and the boundary conditions

The next step is to specify the boundary condition: $u=u_0$ on
$\partial\Omega$. This is done by

```Python
        bc = DirichletBC(V, u0, u0_boundary)
```

where `u0` is an expression defining the solution values on the
boundary, and `u0_boundary` is a function (or object) defining
which points belong to the boundary.

Boundary conditions of the type $u=u_0$ are known as *Dirichlet
conditions*. For the present finite element method for the Poisson
problem, they are also called *essential boundary conditions*, as they
need to be imposed explicitly as part of the trial space (in contrast
to being defined implicitly as part of the variational formulation).
Naturally, the FEniCS class used to define Dirichlet boundary
conditions is named `DirichletBC`.


The variable `u0` refers to an `Expression` object, which is used to
represent a mathematical function. The typical construction is

```Python
        u0 = Expression(formula)
```

where `formula` is a string containing the mathematical expression.
This formula is written with C++ syntax. The expression is
automatically turned into an efficient, compiled C++ function.
The expression may depend on the variables `x[0]` and `x[1]`
corresponding to the $x$ and $y$ coordinates. In 3D, the expression
may also depend on the variable `x[2]` corresponding to the $z$
coordinate. With our choice of $u_0(x,y)=1 + x^2 + 2y^2$, the formula
string can be written as `1 + x[0]*x[0] + 2*x[1]*x[1]`:

```Python
        u0 = Expression('1 + x[0]*x[0] + 2*x[1]*x[1]')
```

[hpl 2: In the code there is a degree parameter that is not included above. Why?]


**String expressions must have valid C++ syntax!**

The string argument to an `Expression` object must obey C++ syntax.
Most Python syntax for mathematical expressions are also valid C++ syntax,
but power expressions make an exception: `p**a` must be written as
`pow(p,a)` in C++ (this is also an alternative Python syntax).
The following mathematical functions can be used directly
in C++
expressions when defining `Expression` objects:
`cos`, `sin`, `tan`, `acos`, `asin`,
`atan`, `atan2`, `cosh`, `sinh`, `tanh`, `exp`,
`frexp`, `ldexp`, `log`, `log10`, `modf`,
`pow`, `sqrt`, `ceil`, `fabs`, `floor`, and `fmod`.
Moreover, the number $\pi$ is available as the symbol `pi`.
All the listed functions are taken from the `cmath` C++ header file, and
one may hence
consult the documentation of `cmath` for more information on the
various functions.

If/else tests are possible using the C syntax for inline branching. The
function

$$
f(x,y) = \left\lbrace\begin{array}{ll} x^2, & x, y\geq 0\\ 
2, & \hbox{otherwise}\end{array}\right.
$$

is implemented as

```Python
        f = Expression('x[0] >= 0 && x[1] >= 0? pow(x[0], 2) : 2')
```

Parameters in expression strings are allowed, but
must be initialized via keyword
arguments when creating the `Expression` object. For example, the
function $f(x)=e^{-\kappa\pi^2t}\sin(\pi k x)$ can be coded as

```Python
        f = Expression('exp(-kappa*pow(pi,2)*t)*sin(pi*k*x[0])',
                       kappa=1.0, t=0, k=4)
```

At any time, parameters can be updated:

```Python
        f.t += dt
        f.k = 10
```

The function `u0_boundary` specifies which points that belong to the
part of the boundary where the boundary condition should be applied:

```Python
        def u0_boundary(x, on_boundary):
            return on_boundary
```

A function like `u0_boundary` for marking the boundary must return a
boolean value: `True` if the given point `x` lies on the Dirichlet
boundary and `False` otherwise.  The argument `on_boundary` is `True`
if `x` is on the physical boundary of the mesh, so in the present
case, where we are supposed to return `True` for all points on the
boundary, we can just return the supplied value of `on_boundary`. The
`u0_boundary` function will be called for every discrete point in the
mesh, which allows us to have boundaries where $u$ are known also
inside the domain, if desired.

One way to think about the specification of boundaries in FEniCS is
that FEniCS will ask you (or rather the function `u0_boundary` which
you have implemented) whether or not a specific point `x` is part of
the boundary. FEniCS already knows whether the point belongs to the
*actual* boundary (the mathematical boundary of the domain) and kindly
shares this information with you in the variable `on_boundary`. You
may choose to use this information (as we do here), or ignore it
completely.

The argument `on_boundary` may also be omitted, but in that case we need
to test on the value of the coordinates in `x`:

```Python
        def u0_boundary(x):
            return x[0] == 0 or x[1] == 0 or x[0] == 1 or x[1] == 1
```

Comparing floating-point values using an exact match test with
`==` is not good programming practice, because small round-off errors
in the computations of the `x` values could make a test `x[0] == 1`
become false even though `x` lies on the boundary.  A better test is
to check for equality with a tolerance, either explicitly

```Python
        def u0_boundary(x):
            return abs(x[0]) < tol or abs(x[1]) < tol \
                or abs((x[0] - 1) < tol or abs(x[1] - 1) < tol
```

or with the `near` command in FEniCS:

```Python
        def u0_boundary(x):
            return near(x[0], 0, tol) or near(x[1], 0, tol) \
                or near(x[0], 1, tol) or near(x[1], 1, tol)
```

### Defining the source term

Before defining the bilinear and linear forms $a(u,v)$ and $L(v)$ we
have to specify the source term $f$:

```Python
        f = Expression('-6')
```

When $f$ is constant over the domain, `f` can be
more efficiently represented as a `Constant`:

```Python
        f = Constant(-6)
```

### Defining the variational problem

We now have all the ingredients we need to define the
variational problem:

```Python
        a = dot(grad(u), grad(v))*dx
        L = f*v*dx
```

In essence, these two lines specify the PDE to be solved.  Note the
very close correspondence between the Python syntax and the
mathematical formulas $\nabla u\cdot\nabla v {\, \mathrm{d}x}$ and $fv {\, \mathrm{d}x}$.  This
is a key strength of FEniCS: the formulas in the variational
formulation translate directly to very similar Python code, a feature
that makes it easy to specify and solve complicated PDE problems.  The
language used to express weak forms is called UFL (Unified Form
Language) [[UFL_2014;@FEniCS]](#UFL_2014;@FEniCS) and is an integral part of FEniCS.

<!-- Instead of `grad` we could also just have written `grad` in the -->
<!-- examples in this tutorial. However, when taking gradients of vector -->
<!-- fields, `grad` and `grad` differ. The latter is consistent with -->
<!-- the tensor algebra commonly used to derive vector and tensor PDEs, -->
<!-- where $\nabla$ ("nabla") acts as a vector operator, and therefore -->
<!-- this author prefers to always use `grad`. -->

### Forming and solving the linear system

Having defined the finite element variational problem and boundary
condition, we can now ask FEniCS to compute the solution:

```Python
        u = Function(V)
        solve(a == L, u, bc)
```

<!-- Some prefer to replace `a` and `L` by an `equation` -->
<!-- variable, which is accomplished by this equivalent code: -->

<!-- !bc pycod -->
<!-- equation = dot(grad(u), grad(v))*dx == f*v*dx -->
<!-- u = Function(V) -->
<!-- solve(equation, u, bc) -->
<!-- !ec -->

Note that we first defined the variable `u` as a `TrialFunction` and
used it to represent the unknown in the form `a`. Thereafter, we
redefined `u` to be a `Function` object representing the solution;
i.e., the computed finite element function $u$. This redefinition of
the variable `u` is possible in Python and often done in FEniCS
applications for linear problems. The two types of objects that `u`
refers to are equal from a mathematical point of view, and hence it is
natural to use the same variable name for both objects.


[AL 3: I AM HERE]

### Examining the values of the solution

The present test problem should produce a numerical solution that
equals the exact solution to machine precision. That is, there are
no approximation errors in our test problem. We can use this property
to "prove" that our implementation is correct, a necessary first step
before we try to apply our code to more complicated problems.
For such verification, we need
to compare the computed `u` function to `u0`.

A finite element function like $u$ is expressed as a linear combination
of basis functions $\phi_j$, spanning the space $V$:

<!-- Equation labels as ordinary links -->
<div id="ftut:poisson1:ufem"></div>

$$
\begin{equation}
u = \sum_{j=1}^N U_j \phi_j \label{ftut:poisson1:ufem} \tag{9}{\thinspace .}
\end{equation}
$$

By writing `solve(a == L, u, bc)` in the program, a linear system
will be formed from $a$ and $L$, and this system is solved for the
$U_1,\ldots,U_N$ values. The $U_1,\ldots,U_N$ values are known
as *degrees of freedom* of $u$. For Lagrange elements (and many other
element types) $U_k$ is simply the value of $u$ at the node
with global number $k$.
The nodes and cell vertices coincide for linear Lagrange elements, while
for higher-order elements there are additional nodes at
the facets and maybe also in the interior of cells.

Having `u` represented as a `Function` object, we can either evaluate
`u(x)` at any point `x` in the mesh (expensive operation!),
or we can grab all the degrees of
freedom values $U_j$ directly by

```Python
        u_nodal_values = u.vector()
```

The result is a `Vector` object, which is basically an
encapsulation of the vector object used in the linear algebra package
that is used to solve the linear system arising from the
variational problem.
Since we program in Python it is convenient to convert the
`Vector` object to a standard `numpy` array for further
processing:

```Python
        u_array = u_nodal_values.array()
```

With `numpy` arrays we can write MATLAB-like code to analyze
the data. Indexing is done with square brackets: `u_array[i]`,
where the index `i` always starts at `0`. However, `i` corresponds
to $u$ at some point in the mesh and the correspondence requires
knowledge of the numbering of degrees of freedom and the numbering of
vertices in elements in the
mesh.


For now, we want to check that the values in `u_array` are correct:
they should equal our `u0` function. The most natural approach is
to interpolate our `u0` expression onto our space
(i.e., the finite element mesh),

```Python
        u0_Function = interpolate(u0, V)
```

The `interpolate` function returns a `Function` object, whose degrees
of freedom values can be obtained by `.vector().array()`.  Our goal is
to show that the degrees of freedom arrays of `u` and `u0_Function`
are equal. One safe of doing this is to compute the maximum error,

```Python
        u0_array = u0_Function.vector().array()  # dof values
        max_error = (u0_array - u.vector().array()).max()
        print('max error:', max_error)
```

**How to check that the error vanishes?**

With inexact arithmetics, as we always have on a computer,
this maximum error is not zero, but should be a small number.
The machine precision is about $10^{-16}$, but in finite element
calculations, rounding errors of this size may accumulate, so
the expected accuracy of `max_error` smaller. Experiments show
that increasing the number of elements and increasing the degree
of the finite element polynomials increase `max_error`.
For a mesh with $2(20\times 20)$ cubic Lagrange elements (degree 3)
`max_error` is about $2\cdot 10^{-12}$, while for 18 linear elements
the maximum error is about $2\cdot 10^{-15}$.



### Plotting the solution

The simplest way of quickly looking at `u` is to say

```Python
        plot(u, interactive=True)
        # or
        plot(u)
        interactive()
```

Clicking on `Help` in the plot windows brings up a list of commands.
For example, typing `m` brings up the mesh.  With the left, middle,
and right mouse buttons you can rotate, translate, and zoom
(respectively) the plotted surface to better examine what the solution
looks like. You must click `Ctrl+q` to kill the plot window and
continue execution beyond the `plot(u, interactive=True)` command or
`interactive()`.

Plotting both the solution and the mesh is accomplished by

```Python
        plot(u)
        plot(mesh)
        # Hold plot
        interactive()
```

Type `Ctrl+w` to kill all plot windows and continue execution.

It is also possible to dump the computed solution to file, e.g., in the
VTK format:

```Python
        vtkfile = File('poisson.pvd')
        vtkfile << u
```

The `poisson.pvd` file can now be loaded into any front-end to VTK,
say ParaView or VisIt. The `plot` function is intended for quick
examination of the solution during program development.  More in-depth
visual investigations of finite element solutions will normally
benefit from using highly professional tools such as ParaView and
VisIt.


Prior to plotting and storing solutions to file it is wise to
give `u` a proper name by `u.rename('u', 'solution')`. Then
`u` will be used as name in plots (rather than the more cryptic
default names like `f_7`).

## Plotting in ParaView

Just to get you quickly started with ParaView, launch the application,
choose **File - Open**, find the file `poisson.pvd`, and click the green **Apply**
button to the left in the GUI. A 2D color plot of $u(x,y)$ is then shown.
You can save the figure to file by **File - Export Scene...** and choosing
a suitable filename.

<!-- dom:FIGURE: [fig/poisson0_paraview.png, weight=800 frac=1] Visualization of test problem in ParaView, with contour lines added in the right plot. -->
<!-- begin figure -->

<p>Visualization of test problem in ParaView, with contour lines added in the right plot.</p>
<img src="fig/poisson0_paraview.png" weight=800>

<!-- end figure -->



# Bibliography

 1. <div id="ArnoldLogg2014"></div> **D. N. Arnold and A. Logg**. 
    Periodic Table of the Finite Elements,
    *SIAM News*,
    2014.

 2. <div id="UFL_2014"></div> **M. S. Alnæs, A. Logg, K. B. Ølgaard, M. E. Rognes and G. N. Wells**. 
    Unified Form Language: A domain-specific language for weak formulations of partial differential equations,
    *ACM Transactions on Mathematical Software*,
    40(2),
    2014,
    doi:10.1145/2566630, arXiv:1211.4047.

 3. <div id="FEniCS"></div> **A. Logg, K.-A. Mardal and G. N. Wells**. 
    Automated Solution of Partial Differential Equations by the Finite Element Method,
    Springer,
    2012.